In [1]:
import numpy as np
import pandas as pd

from Utils.data_proccessing import get_historical_ohlc
from pybit.unified_trading import HTTP
from constants import API_KEY, API_SECRET
from Utils.plotting import plotting_chart_technicals

In [2]:
session = HTTP(demo=True, api_key=API_KEY, api_secret=API_SECRET)
ohlc_m15 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='15')
ohlc_m5 = get_historical_ohlc(session=session, symbol='BTCUSDT', interval='5')

In [3]:
def detect_fvg(last_3_candels):

    fvg = 0
    fvg_top = 0
    fvg_bottom = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if low[-1] > high[0]:

        fvg = 1
        fvg_top = low[-1]
        fvg_bottom = high[0]

    elif high[-1] < low[0]:
        
        fvg = -1
        fvg_top = low[0]
        fvg_bottom = high[-1]

    return fvg, fvg_top, fvg_bottom

In [4]:
def detect_fractal(last_3_candels):
    
    fractal = 0
    fractal_level = 0

    high = last_3_candels['high'].values
    low = last_3_candels['low'].values

    if high[-2] > high[-1] and high[-2] > high[-3]:

        fractal += 1
        fractal_level = high[-2]

    if low[-2] < low[-1] and low[-2] < low[-3]:

        fractal -= 1
        fractal_level = low[-2]

    return fractal, fractal_level

In [5]:
all_fractal = [0,0]
all_fractal_levels = [0,0]

for i in range(2, len(ohlc_m5)):
    
    last_3_candels = ohlc_m5.iloc[i-2:i+1]
    fractal, fractal_level = detect_fractal(last_3_candels)
    all_fractal.append(fractal)
    all_fractal_levels.append(fractal_level)

all_fractal = pd.Series(all_fractal, name='fractal')
all_fractal_levels = pd.Series(all_fractal_levels, name='fractal_level')

ohlc_m5 = pd.concat([ohlc_m5, all_fractal, all_fractal_levels], axis=1)

In [6]:
all_fvg = [0,0]
all_fvg_top = [0,0]
all_fvg_bottom = [0,0]

for i in range(2, len(ohlc_m15)):
    
    last_3_candels = ohlc_m15.iloc[i-2:i+1]
    fvg, fvg_top, fvg_bottom = detect_fvg(last_3_candels)
    all_fvg.append(fvg)
    all_fvg_top.append(fvg_top)
    all_fvg_bottom.append(fvg_bottom)

all_fvg = pd.Series(all_fvg, name='fvg')
all_fvg_top = pd.Series(all_fvg_top, name='fvg_top')
all_fvg_bottom = pd.Series(all_fvg_bottom, name='fvg_bottom')


ohlc_m15 = pd.concat([ohlc_m15, all_fvg, all_fvg_top, all_fvg_bottom], axis=1)

In [7]:
ohlc_m5.tail(10)

,time,open,high,low,close,volume,fractal,fractal_level
2490,2025-10-20 09:15:00,111090.0,111127.8,110680.2,110769.0,926.380,1,111180.0
2491,2025-10-20 09:20:00,110769.0,110839.4,110710.0,110834.4,255.731,-1,110680.2
2492,2025-10-20 09:25:00,110834.4,110834.4,110757.9,110785.8,118.469,0,0.0
2493,2025-10-20 09:30:00,110785.8,110825.1,110671.5,110689.0,157.007,0,0.0
2494,2025-10-20 09:35:00,110689.0,110744.9,110672.3,110700.0,143.548,-1,110671.5
2495,2025-10-20 09:40:00,110700.0,110837.7,110700.0,110818.9,150.499,0,0.0
2496,2025-10-20 09:45:00,110818.9,110837.2,110716.1,110768.8,86.755,1,110837.7
2497,2025-10-20 09:50:00,110768.8,110817.4,110708.4,110813.5,67.754,0,0.0
2498,2025-10-20 09:55:00,110813.5,111050.1,110768.0,110935.3,690.527,-1,110708.4
2499,2025-10-20 10:00:00,110935.3,111068.9,110935.3,111020.3,204.324,0,0.0


In [8]:
ohlc_m15.tail(10)

,time,open,high,low,close,volume,fvg,fvg_top,fvg_bottom
2490,2025-10-20 07:45:00,111205.0,111257.9,111018.9,111148.6,664.703,0,0.0,0.0
2491,2025-10-20 08:00:00,111148.6,111194.5,110999.0,111147.5,640.800,0,0.0,0.0
2492,2025-10-20 08:15:00,111147.5,111235.3,110913.1,111235.3,816.054,0,0.0,0.0
2493,2025-10-20 08:30:00,111235.3,111644.2,111132.3,111275.9,1614.445,0,0.0,0.0
2494,2025-10-20 08:45:00,111275.9,111375.3,111032.0,111061.5,404.609,0,0.0,0.0
2495,2025-10-20 09:00:00,111061.5,111180.0,110923.1,111090.0,483.426,0,0.0,0.0
2496,2025-10-20 09:15:00,111090.0,111127.8,110680.2,110785.8,1300.580,0,0.0,0.0
2497,2025-10-20 09:30:00,110785.8,110837.7,110671.5,110818.9,451.054,-1,110923.1,110837.7
2498,2025-10-20 09:45:00,110818.9,111050.1,110708.4,110935.3,845.036,0,0.0,0.0
2499,2025-10-20 10:00:00,110935.3,111068.9,110935.3,111025.3,200.916,1,110935.3,110837.7


In [9]:
def find_bos_from_fractals(df, mode: str = 'close', max_lookahead=len(ohlc_m5)):
    """
    Повертає копію df з додатковими колонками для рядків-фракталів:
      - fractal_price    : значення fractal_level у рядку фракталу
      - bos_close_price  : close свічки, що зробила BOS (NaN якщо не знайдено)
      - bos_index        : індекс (label) цієї свічки (NaN якщо не знайдено)
      - bos_time         : time цієї свічки (NaN якщо не знайдено)

    mode:
      - 'close' : вважаємо BOS коли close > fractal_level (для fractal==1)
                  або close < fractal_level (для fractal==-1)
      - 'body'  : вважаємо BOS коли мін(open,close) > fractal_level (для fractal==1)
                  або макс(open,close) < fractal_level (для fractal==-1)
    max_lookahead: максимальна кількість свічок у майбутнє (None -> до кінця)
    """
    df = df.copy()

    df['fractal_price'] = np.nan
    df['bos_close_price'] = np.nan
    df['bos_index'] = np.nan
    df['bos_time'] = pd.NaT

    fractal_mask = df['fractal'].notna() & df['fractal_level'].notna()
    fractal_idxs = list(df.index[fractal_mask])

    pos_of_index = {idx: pos for pos, idx in enumerate(df.index)}
    n = len(df)

    for idx in fractal_idxs:
        f = df.at[idx, 'fractal']
        lvl = df.at[idx, 'fractal_level']
        df.at[idx, 'fractal_price'] = lvl

        start_pos = pos_of_index[idx] + 1
        if max_lookahead is None:
            end_pos = n
        else:
            end_pos = min(n, start_pos + int(max_lookahead))

        found = False
        for pos in range(start_pos, end_pos):
            j = df.index[pos]
            o = float(df.iat[pos, df.columns.get_loc('open')])
            c = float(df.iat[pos, df.columns.get_loc('close')])

            if f == 1:
                if mode == 'close':
                    cond = c > lvl
                elif mode == 'body':
                    cond = min(o, c) > lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            elif f == -1:
                if mode == 'close':
                    cond = c < lvl
                elif mode == 'body':
                    cond = max(o, c) < lvl
                else:
                    raise ValueError("mode must be 'close' or 'body'")
            else:
                cond = False

            if cond:
                df.at[idx, 'bos_close_price'] = c
                df.at[idx, 'bos_index'] = j
                df.at[idx, 'bos_time'] = df.at[j, 'time'] if 'time' in df.columns else pd.NaT
                found = True
                break
    return df

ohlc_m5 = find_bos_from_fractals(ohlc_m5, mode='close', max_lookahead=len(ohlc_m5))
ohlc_m5.tail(15)

,time,open,high,low,close,volume,fractal,fractal_level,fractal_price,bos_close_price,bos_index,bos_time
2485,2025-10-20 08:50:00,111124.0,111141.4,111033.0,111033.1,153.004,1,111375.3,111375.3,NaN,NaN,NaT
2486,2025-10-20 08:55:00,111033.1,111119.8,111032.0,111061.5,98.947,0,0.0,0.0,NaN,NaN,NaT
2487,2025-10-20 09:00:00,111061.5,111067.1,110954.0,110977.1,188.632,0,0.0,0.0,NaN,NaN,NaT
2488,2025-10-20 09:05:00,110977.1,111099.5,110923.1,111076.0,182.079,0,0.0,0.0,NaN,NaN,NaT
2489,2025-10-20 09:10:00,111076.0,111180.0,111069.0,111090.0,112.715,-1,110923.1,110923.1,110769.0,2490.0,2025-10-20 09:15:00
2490,2025-10-20 09:15:00,111090.0,111127.8,110680.2,110769.0,926.380,1,111180.0,111180.0,NaN,NaN,NaT
2491,2025-10-20 09:20:00,110769.0,110839.4,110710.0,110834.4,255.731,-1,110680.2,110680.2,NaN,NaN,NaT
2492,2025-10-20 09:25:00,110834.4,110834.4,110757.9,110785.8,118.469,0,0.0,0.0,NaN,NaN,NaT
2493,2025-10-20 09:30:00,110785.8,110825.1,110671.5,110689.0,157.007,0,0.0,0.0,NaN,NaN,NaT
2494,2025-10-20 09:35:00,110689.0,110744.9,110672.3,110700.0,143.548,-1,110671.5,110671.5,NaN,NaN,NaT


In [10]:
def find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead, debug: bool = False):
    """
    Шукає пари фракталів після M15 FVG і першу M5-свічку ПО ЧАСУ після f2,
    у якої close > f1_price (long) або close < f1_price (short).
    Повертає (signals_df, m5_out) — signals і m5 з мітками (в оригінальному індексі).
    """

    m15 = ohlc_m15.copy()
    m5_orig = ohlc_m5.copy()
    m15['time'] = pd.to_datetime(m15['time'])
    m5_orig['time'] = pd.to_datetime(m5_orig['time'])

    m5_orig = m5_orig.reset_index().rename(columns={'index': '_orig_index'})

    # Робоча версія з позиційним індексом 0..N-1
    m5 = m5_orig.sort_values('time').reset_index(drop=True)

    m5['is_f1'] = 0
    m5['is_f2'] = 0
    m5['is_bos'] = 0
    m5['source_fvg_time'] = pd.NaT

    signals = []
    n_m5 = len(m5)

    for _, fvg in m15[m15['fvg'] != 0].iterrows():
        fvg_dir = int(fvg['fvg'])
        fvg_time = fvg['time']
        fvg_top = float(fvg['fvg_top'])
        fvg_bottom = float(fvg['fvg_bottom'])

        m5_after = m5[m5['time'] > fvg_time]
        if m5_after.empty:
            continue

        # список фракталів (тільки реальні: !=0 і не NaN)
        fracs = m5_after[(m5_after['fractal'] != 0) & (~m5_after['fractal'].isna())]
        if len(fracs) < 2:
            continue

        frac_positions = list(fracs.index)

        for i in range(len(frac_positions) - 1):
            p1 = frac_positions[i]
            p2 = frac_positions[i + 1]
            r1 = m5.loc[p1]
            r2 = m5.loc[p2]

            if fvg_dir == 1:
                # long: f1 == 1 та f1.price > fvg_top
                #       f2 == -1 та f2.price < fvg_top і > fvg_bottom
                cond_pair = (r1['fractal'] == 1) and (float(r1['fractal_level']) > fvg_top) \
                            and (r2['fractal'] == -1) and (float(r2['fractal_level']) < fvg_top) and (float(r2['fractal_level']) > fvg_bottom)
            else:
                # short: f1 == -1 та f1.price < fvg_top
                #        f2 == 1 та f2.price > fvg_top і < fvg_bottom
                cond_pair = (r1['fractal'] == -1) and (float(r1['fractal_level']) < fvg_bottom) \
                            and (r2['fractal'] == 1) and (float(r2['fractal_level']) > fvg_bottom) and (float(r2['fractal_level']) < fvg_top)

            if not cond_pair:
                continue

            start_pos = p2
            if start_pos >= n_m5:
                break

            f1_price = float(r1['fractal_level'])
            bos_pos = None

            end_pos = n_m5 if max_lookahead is None else min(n_m5, start_pos + int(max_lookahead))

            for pos in range(start_pos, end_pos):
                close = float(m5.at[pos, 'close'])
                if fvg_dir == 1:
                    cond = close > f1_price   # long: close вище f1
                elif fvg_dir == -1:
                    cond = close < f1_price   # short: close нижче f1
                else:
                    return None

                if cond:
                    bos_pos = pos
                    break

            if bos_pos is not None:
                sig = {
                    'fvg_dir': fvg_dir,
                    'fvg_time': fvg_time,
                    'fvg_top': fvg_top,
                    'fvg_bottom': fvg_bottom,
                    'f1_pos': int(p1),
                    'f1_time': r1['time'],
                    'f1_price': float(r1['fractal_level']),
                    'f2_pos': int(p2),
                    'f2_time': r2['time'],
                    'f2_price': float(r2['fractal_level']),
                    'bos_pos': int(bos_pos),
                    'bos_time': m5.at[bos_pos, 'time'],
                    'bos_close': float(m5.at[bos_pos, 'close'])
                }
                signals.append(sig)

                m5.at[p1, 'is_f1'] = 1
                m5.at[p2, 'is_f2'] = 1
                m5.at[bos_pos, 'is_bos'] = 1
                m5.at[p1, 'source_fvg_time'] = fvg_time
                m5.at[p2, 'source_fvg_time'] = fvg_time
                m5.at[bos_pos, 'source_fvg_time'] = fvg_time
                
    signals_df = pd.DataFrame(signals)
    if '_orig_index' in m5.columns:
        m5_out = m5.set_index('_orig_index').sort_index()
    else:
        if debug:
            print("Warning: '_orig_index' not in m5.columns; returning positional-index m5")
        m5_out = m5

    return signals_df, m5_out

signals, m5_annot = find_fvg_fractal_bos_by_close(ohlc_m15, ohlc_m5, max_lookahead=len(ohlc_m5))

signals.tail()

,fvg_dir,fvg_time,fvg_top,fvg_bottom,f1_pos,f1_time,f1_price,f2_pos,f2_time,f2_price,bos_pos,bos_time,bos_close
1025,-1,2025-10-19 19:45:00,108987.2,108917.8,2332,2025-10-19 20:05:00,108740.0,2333,2025-10-19 20:10:00,108937.1,2345,2025-10-19 21:10:00,108723.7
1026,-1,2025-10-19 19:45:00,108987.2,108917.8,2340,2025-10-19 20:45:00,108759.3,2341,2025-10-19 20:50:00,108952.2,2345,2025-10-19 21:10:00,108723.7
1027,1,2025-10-19 22:15:00,108974.7,108745.8,2365,2025-10-19 22:50:00,109293.8,2370,2025-10-19 23:15:00,108755.5,2425,2025-10-20 03:50:00,110270.2
1028,1,2025-10-20 04:00:00,110047.0,109221.0,2431,2025-10-20 04:20:00,110433.2,2433,2025-10-20 04:30:00,109878.8,2441,2025-10-20 05:10:00,110487.2
1029,1,2025-10-20 06:00:00,110950.0,110763.0,2479,2025-10-20 08:20:00,111200.0,2480,2025-10-20 08:25:00,110913.1,2480,2025-10-20 08:25:00,111235.3


In [11]:
from main import qty_from_risk

def qty_from_risk(entry_price, stop_loss, risk_usd=1000):
    dist = abs(entry_price - stop_loss)
    if dist <= 0:
        return 0
    qty = risk_usd / dist
    return round(qty, 3)

def calc_pnl(entry_price, exit_price, stop_loss, risk_usd=1000, direction=1):
    qty = qty_from_risk(entry_price, stop_loss, risk_usd)

    price_diff = (exit_price - entry_price) * direction

    pnl = qty * price_diff
    return pnl, qty


def run_backtest_signals(signals_df, m5_df, rr=2.5, max_lookahead=None,
                         conservative_same_candle=True, strict_entry_candle=False):
    """
    Як раніше, але додає перевірку: якщо TP/SL були торкнуті **до** фактичної
    свічки входу (у проміжку від BOS до entry_idx-1), сигнал пропускається.
    strict_entry_candle: якщо True - і в самій свічці входу є торк TP/SL -> пропускаємо
    (бо не можемо відрізнити порядок подій).
    """
    if signals_df is None or len(signals_df) == 0:
        return pd.DataFrame()

    m5 = m5_df.copy()
    m5['time'] = pd.to_datetime(m5['time'])
    m5 = m5.sort_values('time').reset_index(drop=True)

    trades = []
    last_idx = len(m5) - 1

    for _, sig in signals_df.iterrows():
        try:
            fvg_dir = int(sig.get('fvg_dir', 0))
        except Exception:
            continue
        if fvg_dir == 0:
            continue

        f1_price = float(sig['f1_price'])
        f2_price = float(sig['f2_price'])

        if 'bos_time' in sig and pd.notnull(sig['bos_time']):
            bos_time = pd.to_datetime(sig['bos_time'])
            start_loc = int(m5['time'].searchsorted(bos_time, side='right'))
        elif 'bos_pos' in sig and not pd.isna(sig['bos_pos']):
            pos = int(sig['bos_pos'])
            start_loc = pos + 1
        else:
            continue

        if start_loc >= len(m5):
            continue

        direction = 1 if fvg_dir == 1 else -1
        entry_price = f1_price
        sl = f2_price
        if direction == 1:
            risk = entry_price - sl
            tp = entry_price + risk * rr
        else:
            risk = sl - entry_price
            tp = entry_price - risk * rr

        if abs(risk) < 1e-12:
            continue

        if max_lookahead is not None:
            search_end = min(last_idx, start_loc + int(max_lookahead) - 1)
        else:
            search_end = last_idx

        entry_idx = None

        for idx in range(start_loc, search_end + 1):
            row = m5.loc[idx]
            high = float(row['high'])
            low = float(row['low'])

            if low <= f1_price <= high:
                prior_touched = False
                if idx > start_loc:
                    prior_slice = m5.loc[start_loc:idx-1]
                    if direction == 1:
                        if (prior_slice['high'].astype(float) >= tp).any() or (prior_slice['low'].astype(float) <= sl).any():
                            prior_touched = True
                    else:
                        if (prior_slice['low'].astype(float) <= tp).any() or (prior_slice['high'].astype(float) >= sl).any():
                            prior_touched = True

                if prior_touched:
                    entry_idx = None
                    break  
                else:
                    entry_candle_touched = False
                    if strict_entry_candle:
                        if direction == 1:
                            if (high >= tp) or (low <= sl):
                                entry_candle_touched = True
                        else:
                            if (low <= tp) or (high >= sl):
                                entry_candle_touched = True
                        if entry_candle_touched:
                            entry_idx = None
                            break

                    entry_idx = idx
                    break

        if entry_idx is None:
            continue

        if max_lookahead is not None:
            end_idx = min(last_idx, entry_idx + int(max_lookahead) - 1)
        else:
            end_idx = last_idx

        exit_idx = None
        exit_price = None
        result = None
        pnl = None

        for idx in range(entry_idx, end_idx + 1):
            row = m5.loc[idx]
            high = float(row['high'])
            low = float(row['low'])

            if direction == 1:
                touched_sl = low <= sl
                touched_tp = high >= tp

                if touched_sl and touched_tp:
                    if conservative_same_candle:
                        exit_idx = idx; exit_price = sl; result = 'loss'; pnl = sl - entry_price; break
                    else:
                        exit_idx = idx; exit_price = tp; result = 'win'; pnl = tp - entry_price; break
                elif touched_sl:
                    exit_idx = idx; exit_price = sl; result = 'loss'; pnl = sl - entry_price; break
                elif touched_tp:
                    exit_idx = idx; exit_price = tp; result = 'win'; pnl = tp - entry_price; break
            else:
                touched_sl = high >= sl
                touched_tp = low <= tp

                if touched_sl and touched_tp:
                    if conservative_same_candle:
                        exit_idx = idx; exit_price = sl; result = 'loss'; pnl = entry_price - sl; break
                    else:
                        exit_idx = idx; exit_price = tp; result = 'win'; pnl = entry_price - tp; break
                elif touched_sl:
                    exit_idx = idx; exit_price = sl; result = 'loss'; pnl = entry_price - sl; break
                elif touched_tp:
                    exit_idx = idx; exit_price = tp; result = 'win'; pnl = entry_price - tp; break

        if exit_idx is None:
            exit_idx = end_idx
            exit_price = float(m5.loc[end_idx, 'close'])
            pnl = (exit_price - entry_price) if direction == 1 else (entry_price - exit_price)
            result = 'timeout'


        qty = qty_from_risk(entry_price, sl, risk_usd=1000)
        pnl = (exit_price - entry_price) * direction * qty

        trades.append({
            'signal_index': sig.name if sig.name is not None else None,
            'direction': direction,
            'fvg_time': pd.to_datetime(sig.get('fvg_time', pd.NaT)),
            'fvg_top': float(sig.get('fvg_top', np.nan)),
            'fvg_bottom': float(sig.get('fvg_bottom', np.nan)),
            'f1_time': pd.to_datetime(sig.get('f1_time', pd.NaT)),
            'f1_price': float(sig.get('f1_price', np.nan)),
            'f2_time': pd.to_datetime(sig.get('f2_time', pd.NaT)),
            'f2_price': float(sig.get('f2_price', np.nan)),
            'entry_time': m5.loc[entry_idx, 'time'],
            'entry_index': int(entry_idx),
            'entry_price': entry_price,
            'sl': sl,
            'tp': tp,
            'exit_time': m5.loc[int(exit_idx), 'time'],
            'exit_index': int(exit_idx),
            'exit_price': exit_price,
            'result': result,
            'pnl': round(pnl, 0)
        })

    trades_df = pd.DataFrame(trades)

    return trades_df

trades = run_backtest_signals(signals, m5_annot, rr=2.5, max_lookahead=20, conservative_same_candle=True)


In [12]:
trades.tail()

,signal_index,direction,fvg_time,fvg_top,fvg_bottom,f1_time,f1_price,f2_time,f2_price,entry_time,entry_index,entry_price,sl,tp,exit_time,exit_index,exit_price,result,pnl
889,1024,-1,2025-10-19 19:15:00,109229.7,109152.4,2025-10-19 22:00:00,108575.6,2025-10-19 22:20:00,109194.7,2025-10-19 23:50:00,2377,108575.6,109194.7,107027.85,2025-10-20 01:25:00,2396,108074.4,timeout,809.0
890,1025,-1,2025-10-19 19:45:00,108987.2,108917.8,2025-10-19 20:05:00,108740.0,2025-10-19 20:10:00,108937.1,2025-10-19 21:35:00,2350,108740.0,108937.1,108247.25,2025-10-19 22:05:00,2356,108937.1,loss,-1000.0
891,1026,-1,2025-10-19 19:45:00,108987.2,108917.8,2025-10-19 20:45:00,108759.3,2025-10-19 20:50:00,108952.2,2025-10-19 21:35:00,2350,108759.3,108952.2,108277.05,2025-10-19 22:05:00,2356,108952.2,loss,-1000.0
892,1028,1,2025-10-20 04:00:00,110047.0,109221.0,2025-10-20 04:20:00,110433.2,2025-10-20 04:30:00,109878.8,2025-10-20 05:15:00,2442,110433.2,109878.8,111819.20,2025-10-20 06:50:00,2461,111239.8,timeout,1455.0
893,1029,1,2025-10-20 06:00:00,110950.0,110763.0,2025-10-20 08:20:00,111200.0,2025-10-20 08:25:00,110913.1,2025-10-20 08:40:00,2483,111200.0,110913.1,111917.25,2025-10-20 09:15:00,2490,110913.1,loss,-1000.0


In [13]:
from Utils.metrics import calculate_metrics

all_results = []
trades_data = run_backtest_signals(signals, m5_annot, rr=2.5, max_lookahead=20, conservative_same_candle=True)
trades_data.to_csv('trades_data.csv', index=False)
profits = trades_data['pnl']
results_metrics = calculate_metrics(profits)
all_results.append(results_metrics)

all_results = pd.concat(all_results, axis=0)
all_results.to_csv('all_results.csv', index=False)
all_results


,Total Trades,Total Net Profit,Average Profit,Win rate,Average Win,Average Loss,Profit Factor,Max Consecutive Losses,Max Drawdown
0,894,14346.0,16.05,0.39,1381.29,-854.11,1.03,18,53164.0
